# Integracia dat

<!--
navody na pouzivanie pandas, matplotlib a numpy na spracovanie dat. Niesu to informacie o tom ako robit explorativnu analyzu, ale ako pouzivat kniznice

Z tohoto povyberam zaujimave casti, spojim ich s nejakou kapitolou v knihe o tom ako riesit spracovanie, cistanie dat a transformovanie dat
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_10_pandas_introduction.ipynb
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_11_pandas_adding_data.ipynb
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_12_pandas_groupby.ipynb
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_13_pandas_movies.ipynb 
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_14_pandas_reshape.ipynb
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_15_pandas_transforming.ipynb
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_21_pandas_processing.ipynb
http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_22_pandas_cleaning.ipynb

http://nbviewer.jupyter.org/github/ResearchComputing/Meetup-Fall-2013/blob/master/python/lecture_23_titanic_example.ipynb
-->

In [ ]:
from IPython.display import Image
Image('ML_Workflow.PNG')

# O com nejdem hovorit

* Nejdem opisovat vsetky mozne record linkage a entity mapping metody (to je minimalne na samostatnu prednasku)
* Nejdem opisovat komplexne ETL nastroje a postupy na spajanie tabuliek a roznych databaz (na to tu mame dokonca samostatny predmet)

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Skusme sa pohrat s nejakymi datami

In [ ]:
%%bash
head data/BETR8010000800100hour.1-1-1990.31-12-2012

z tohoto zatial vieme asi len to, ze pojde o csv format, separator hodnot je \t, su tam same numericke data a nemame pomenovane atributy

In [ ]:
data = pd.read_csv("data/BETR8010000800100hour.1-1-1990.31-12-2012", sep='\t')
data.head()
# Data su tvorene meraniami nejakej veliciny v jednotlivych hodinach dna. 
# Co den, to riadok. Kazda hodina ma zvlast stlpec + je tu stlpec pre nejaky flag, ktory nas nezaujima
# su tam nejak divne hodnoty, ktore by tam asi nemali byt -999 a -9999
# datum bude asi index
# v subore nieje hlavicka

In [ ]:
filename = "data/BETR8010000800100hour.1-1-1990.31-12-2012"

data = pd.read_csv(filename, sep='\t', header=None,
                   na_values=[-999, -9999], index_col=0)
# vela upratovania dat vieme spravit uz pri nacitani
data.head()

In [ ]:
# skusime povyhadzovat tie flagy, ktore nas nezaujimaju. Zhodou okolnosti je to kazdy druhy stlpec
data.columns[1::2]

In [ ]:
data = data.drop(data.columns[1::2], axis=1)
data.head()

In [ ]:
# Skusme si nejak normalne poemnvoat vzniknute stlpce
["{:02d}".format(i) for i in range(len(data.columns))]

In [ ]:
# mam nejako rozsypane nazvy stlpcov
data.columns = ["{:02d}".format(i) for i in range(len(data.columns))]
data.head()

In [ ]:
data = data.stack()
data.head()

In [ ]:
type(data) # vysledok preusporiadania je viacdimenzionaly Series objekt a nie DataFrame. Ja chcem mat pekny data frame, tak s tim nieco spravime

In [ ]:
# mohli by sme nejak normalne pomenovat stlpec
# napriklad nazvom stanice
import os
_, fname = os.path.split(filename)
station = fname[:7]
print(filename)
print(station)

In [ ]:
data = data.reset_index(name=station) #reset index mi z toho sprav data frame
# data = data.reset_index() #reset index mi z toho sprav data frame
print(type(data))
data.head()

In [ ]:
data = data.rename(columns = {0:'date', 'level_1':'hour'})
data.head()

In [ ]:
# teraz tomu vyrobime novy index z datumu a hodiny
data.index = pd.to_datetime(data['date'] + ' ' + data['hour'])
data.head()

In [ ]:
# a zmazeme nepotrebne stlpce
data = data.drop(['date', 'hour'], axis=1)
data.head()
# Teraz uz mame data, s ktorymi sa uz da nieco robit

Ja mam tych suborov viac. Kazdy obsahuje data z inej meracej stanice. Aby som zjednodusil prezentaciu, tak predchadzajuci kod som dal do cyklu a vlozil do skriptu

In [ ]:
import airbase
no2 = airbase.load_data()

In [ ]:
no2.head(3)

In [ ]:
no2.tail()

In [ ]:
no2.info()

In [ ]:
no2.describe()

In [ ]:
no2.plot(kind='box')

In [ ]:
# boxplot vie ukazat aj outlierov
sns.boxplot(no2, sym='k.')

tych outlierov by som mal normalne vidiet aj v tom klasickom boxplote, ale je nejaky bug vo verzii matplotlibu, ktoru mam nainstalovanu a zle interaguje s importovanou kniznicou seaborn. Preto som ich radsej ukazal v kniznici seaborn. Ak by som ju ale vobec nenaimportotavl, tak by tam tie outliery mali byt. 
Vraj je to uz v najnovsej verzii matplotlibu opravene :(

In [ ]:
no2['BETN029'].plot(kind='hist', bins=50)

In [ ]:
import seaborn

In [ ]:
seaborn.violinplot(no2)

In [ ]:
no2.plot(figsize=(12,6))
# mozem si vyplotovat surove data, ale je otazne, co mi to povie

In [ ]:
# mozem si povedat, ze chcem len nejaku mensiu cast
no2[-500:].plot(figsize=(12,6))

alebo pouzijem zaujimavejsie operacie s casovymi radmi

In [ ]:
no2.index # kedze index su casy, tak viem robit s nimi zaujimave veci

In [ ]:
no2["2010-01-01 09:00": "2010-01-01 12:00"] # napriklad definovat rozsahy pomocou stringu s datumom

In [ ]:
no2['2012'] # alebo takto vybrat vsetky data z jedneho konkretneho roku
# no2['2012'].head()
# no2['2012/03'] # alebo len data z marca

In [ ]:
# komponenty datumu su pristupne z indexu
# no2.index.hour
no2.index.year

In [ ]:
# a co je zaujimavejsie viem zmenit vzorkovaciu frekvenciu
no2.resample('D').mean().head()

In [ ]:
no2.resample('M').mean().plot()
# toto sa zda, ze povie uz trochu viac. Napriklad, ze je tu asi nejaka sezonnost

In [ ]:
no2.resample('A').mean().plot()
# a mozno aj nejaky dlhodoby trend

In [ ]:
no2['2012-3':'2012-4'].resample('D').mean().plot()
# mozno je tam aj nejaka tyzdenna sezonnost

In [ ]:
# mozem pouzit aj viacero agregacnych funkcii a porovnat si ich
no2.loc['2009':, 'FR04037'].resample('M').agg(['mean', 'median']).plot()
# no2.loc['2009':, 'FR04037'].resample('M').agg(['mean', 'std']).plot()

## Pozor resample != groupby

In [ ]:
no2.resample('M').mean().plot()

In [ ]:
no2.groupby(no2.index.month).mean().plot()

# Zopar uloh na groupby

Otazka: ako by ste vyplotovali typycky denny priebeh hodnoty pre rozne stanice?
<!--
no2.groupby(no2.index.hour).mean().plot()
-->

Otazka: aky je rozdiel v priebehu hodnot medzi typickym dnom v tyzdni a cez vikend pre stanicu FR04012?
<!--
no2['weekday'] = no2.index.weekday
no2['weekend'] = no2['weekday'].isin([5, 6])
data_weekend = no2.groupby(['weekend', no2.index.hour]).mean()
data_weekend_FR04012 = data_weekend['FR04012'].unstack(level=0)
data_weekend_FR04012.plot()
-->

# Priklad analyzy s pouzitim ineho datasetu
tentokrat to nebudu casove rady, ale klasicky dataset na predvadzanie kalsifikacie Iris

In [ ]:
iris_data = pd.read_csv('data/iris-data.csv')
iris_data.head()
# toto je trochu spotvoreny dataset kvetiniek

In [ ]:
iris_data.info()

In [ ]:
iris_data.describe()

In [ ]:
seaborn.pairplot(iris_data.dropna(), hue='class')

In [ ]:
iris_data.loc[iris_data['class'] == 'versicolor', 'class'] = 'Iris-versicolor'
iris_data.loc[iris_data['class'] == 'Iris-setossa', 'class'] = 'Iris-setosa'

iris_data['class'].unique()

In [ ]:
seaborn.pairplot(iris_data.dropna(), hue='class')

In [ ]:
iris_data.loc[iris_data['class'] == 'Iris-versicolor', 'sepal_length_cm'].hist()

In [ ]:
plt.rc("lines", markeredgewidth=0.5)
iris_data.loc[iris_data['class'] == 'Iris-versicolor', 'sepal_length_cm'].plot(kind='box')

In [ ]:
iris_data.loc[(iris_data['class'] == 'Iris-versicolor') & (iris_data['sepal_length_cm'] < 1 ), 'sepal_length_cm']

In [ ]:
iris_data.loc[(iris_data['class'] == 'Iris-versicolor') & (iris_data['sepal_length_cm'] > 1 ), 'sepal_length_cm']

In [ ]:
mask = (iris_data['class'] == 'Iris-versicolor') & (iris_data['sepal_length_cm'] < 1 )

iris_data.loc[mask, 'sepal_length_cm'] = iris_data.loc[mask, 'sepal_length_cm'] * 100

In [ ]:
iris_data.loc[mask, 'sepal_length_cm']

In [ ]:
seaborn.pairplot(iris_data.dropna(), hue='class')

## Skusme sa pozriet este na tie chybajuce hodnoty

In [ ]:
iris_data.loc[(iris_data['sepal_length_cm'].isnull()) |
              (iris_data['sepal_width_cm'].isnull()) |
              (iris_data['petal_length_cm'].isnull()) |
              (iris_data['petal_width_cm'].isnull())]

In [ ]:
iris_data.loc[iris_data['class'] == 'Iris-setosa', 'petal_width_cm'].hist()

In [ ]:
average_petal_width = iris_data.loc[iris_data['class'] == 'Iris-setosa', 'petal_width_cm'].mean()

iris_data.loc[(iris_data['class'] == 'Iris-setosa') &
              (iris_data['petal_width_cm'].isnull()),
              'petal_width_cm'] = average_petal_width

In [ ]:
seaborn.pairplot(iris_data, hue='class')

## Iny sposob nahradzania prazdnych hodnot
Pozor, toto je len najjednoduchsia moznost, kde nahradzam chybajucu hodnotu nejakou konstantou. Existuje velmi vela dalsich metod, ktore by ste mali pouzit aj vo svojom projekte.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rasbt/python_reference/master/Data/some_soccer_data.csv')
df.head()

In [ ]:
# predtym sme to robili manulane. 
# iris_data.loc[(iris_data['class'] == 'Iris-setosa') & (iris_data['petal_width_cm'].isnull()), 'petal_width_cm'] = average_petal_width

# Da sa na to pouzit takato pekna funkcia
df.fillna(value=0, inplace=True)
df

## Existuje vsak este elegantnejsi sposob

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rasbt/python_reference/master/Data/some_soccer_data.csv')
df = df.rename(columns={'P': 'points', 
                        'GP': 'games',
                        'SOT': 'shots_on_target',
                        'G': 'goals',
                        'PPG': 'points_per_game',
                        'A': 'assists',})
df['SALARY'] = df['SALARY'].apply(lambda x: x.strip('$m'))

def process_player_col(text):
    name, rest = text.split('\n')
    position, team = [x.strip() for x in rest.split(' — ')]
    return pd.Series([name, team, position])

df[['PLAYER', 'team', 'position']] = df.PLAYER.apply(process_player_col)            
               
df.head()

In [ ]:
# v kniznici sklearn uz su predpripravene nejake metody na nahradzanie chybajucich hodnot
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
df[['games', 'assists']] = imp.fit_transform(df[['games', 'assists']].values)
df.head()

Pozor, toto doplnanie neberie do uvahy triedu

In [ ]:
df.games.mean()

In [ ]:
df[df.position == 'Forward'].games.mean()

Cize mozno by bolo lepsie spravit trochu inteligentnejsie nahradzanie chybajucich hodnot, ktore pocita oddelene priemery pre rozne skupiny hracov. Toto sa vola nahradenie chýbajúcej hodnoty priemerom segmentu (je to jedna z moznosti, ktoru by ste mali vyskusat v druhej casti projektu).

# Sumar co si zobrat z tejto explorativnej analyzy

* Uisite sa, ze data su kodovane spravne (najcastejsie sa treba pozriet manualne do dat)
* Uistite sa, ze data spadaju do ocakavaneho rozsahu a vsetky maju ocakavany tvar (napriklad format casu)
* Porieste chybajuce data napriklad vyhodenim, nahradenim priemerom (priemer moze byt s ohladom na triedu), ...
* Nikdy nesahajte do dat manualne. Vzdy pouzivajte kod, ktory si odlozite a pouzijete vzdy ked budete opakovat experiment. Chceme aby bola analyza reprodukovatelna
* Spravte si grafy vsetkeho, co sa len da, aby ste si vizualne potvrdili, ze nieco je tak ako by malo byt